In [12]:
import os
import csv
import re

import pandas as pd

In [13]:
previous_files = [x for x in os.listdir("../data/Scraped data/") if ".parquet" in x]

if previous_files:
    sorted_files = sorted(
        previous_files, key=lambda x: int(re.search(r"\d+", x).group())
    )
    latest_file = sorted_files[-1]

    latest_version = int(
        max([file_name.split("v")[1].split(".")[0] for file_name in previous_files])
    )
    new_version = latest_version + 1
else:
    new_version = 1

In [14]:
# Define path to data
path_data = "../data/Scraped data/June 2023"

# empty list to store data
data_list = []

# loop through files in directory
for file in os.listdir(path_data):
    # check if file is in list of files to load
    if ".csv" in file:
        # open file and read data
        with open(os.path.join(path_data, file), newline='') as f:
            # create csv reader object
            reader = csv.reader(f)
            # iterate over rows in the csv file and add filename to each row
            reader_data = [row + [file.split(".")[0]] for row in reader]
            # store column names and data in data_list
            col_names = reader_data[0]
            data_list.append(reader_data[1:])

# concatenate data from all files into one dataframe
data = pd.concat([pd.DataFrame(data) for data in data_list]).reset_index(drop=True)

# Drop the first (index as a column) and last column (postcode - the above code must
# have added it)
data.drop(columns=[0, 11], inplace=True)
data.columns = col_names[1:11]

# Drop duplicates
data = data.drop_duplicates(subset="carId").reset_index(drop=True)

# print first five rows of data
data.head()


,model,year,price,transmission,mileage,fuelType,mpg,engineSize,brand,carId
0,T-Cross,2021,19990,Manual,8323,Petrol,49.6,1,volkswagen,29898913
1,Golf,2022,21991,Manual,8811,Petrol,52.3,1,volkswagen,29856808
2,T-Roc,2021,22691,Manual,9001,Petrol,46.3,1,volkswagen,29799830
3,T-Roc,2019,23992,Manual,9196,Diesel,50.4,1.6,volkswagen,29676122
4,Golf,2020,30994,Semi-Auto,10882,Hybrid,176.6,1.4,volkswagen,29482261


In [15]:
# Remove cars of fuel type equal to 'Petrol/Electric'
data = data[data["fuelType"] != "Petrol/Electric"]
data.reset_index(drop=True, inplace=True)

# Create a boolean ev_mask for elements containing the word "Electric"
ev_mask = data.apply(lambda x: x.str.contains('Electric', case=False)).any(axis=1)

ev_mileage = data.loc[ev_mask, "mpg"]
data = data.copy()
data.loc[ev_mask, "mileage"] = ev_mileage

# Clean data from electric cars. Because some fields such as mpg or engineSize doesn't
# really apply to them, the scraping tool didn't get the information right
data.loc[ev_mask, "transmission"] = "Automatic"
data.loc[ev_mask, "mpg"] = 0
data.loc[ev_mask, "fuelType"] = "Electric"
data.loc[ev_mask, "engineSize"] = 0

data[ev_mask].sample(10)

,model,year,price,transmission,mileage,fuelType,mpg,engineSize,brand,carId
3668,ID3,2023,34990,Automatic,3621,Electric,0,0,volkswagen,29440790
11666,i3,2017,17950,Automatic,18196,Electric,0,0,bmw,29786131
26311,Golf,2016,13999,Automatic,39515,Electric,0,0,volkswagen,29303435
3824,ID3,2020,24980,Automatic,16563,Electric,0,0,volkswagen,30053591
3783,ID3,2020,25795,Automatic,23880,Electric,0,0,volkswagen,26415057
3686,ID5,2022,45990,Automatic,5000,Electric,0,0,volkswagen,29287734
17507,Q4 e-tron,2021,45997,Automatic,10361,Electric,0,0,audi,29670945
3632,ID3,2020,24110,Automatic,7376,Electric,0,0,volkswagen,28896148
8,ID3,2020,24999,Automatic,14656,Electric,0,0,volkswagen,28818198
3579,Up,2022,18989,Automatic,4915,Electric,0,0,volkswagen,29290838


In [16]:
# Turn the mileage column into numeric type and remove NaN values
data["mileage"] = pd.to_numeric(data['mileage'], errors='coerce')
data = data.dropna()
data.reset_index(drop=True, inplace=True)

# Create a mask to identify where a value in any columns is equal to "None"
columns_to_apply = data.columns[data.columns != "mileage"]
none_mask = (
    data[columns_to_apply]
    .apply(lambda x: x.str.contains("None", case=False))
    .any(axis=1)
)
# Create a mask to identify where the mileage column is an integer - float values need
# be removed
int_mask = data["mileage"].apply(lambda x: x.is_integer())

# Apply the masks to remove float values in mileage and any values equal to "None"
data = data[(~none_mask) & (int_mask)]
data.reset_index(drop=True, inplace=True)

In [17]:
# Change column type of the columns that are meant to be numeric to int and float types
int_cols = ["year", "price", "mileage"]
float_cols = ["mpg", "engineSize"]
data[int_cols] = data[int_cols].astype(int)
data[float_cols] = data[float_cols].astype(float)

# Round engine size to 1 decimal figure
data["engineSize"] = round(data["engineSize"],1)

# Make sure that categorical columns only contain the right categories
# Fuel type
data = data.loc[
    (data["fuelType"] == "Petrol")
    | (data["fuelType"] == "Diesel")
    | (data["fuelType"] == "Hybrid")
    | (data["fuelType"] == "Electric")
]
data.reset_index(drop=True, inplace=True)
# Transmission
data = data.loc[
    (data["transmission"] == "Automatic")
    | (data["transmission"] == "Manual")
    | (data["transmission"] == "Semi-Auto")
]
data.reset_index(drop=True, inplace=True)

# Replace the brand "volkswagen" with "vw" as this used in the other dataset
data.loc[data["brand"] == "volkswagen", "brand"] = "vw"

data_save = data.copy()
data_save.drop(columns=["carId"], inplace=True)
data_save.to_csv(f"../data/Scraped data/all_scraped_cars-v{new_version}.csv")
data_save.to_parquet(f"../data/Scraped data/all_scraped_cars-v{new_version}.parquet")



In [19]:
data.sample(10)

,model,year,price,transmission,mileage,fuelType,mpg,engineSize,brand,carId
12718,Golf,2012,8995,Manual,56190,Petrol,44.1,1.4,vw,29935126
8676,3 Series,2022,38990,Semi-Auto,5435,Diesel,52.3,2.0,bmw,30004885
10823,Q5,2013,10295,Automatic,137558,Diesel,47.1,2.0,audi,28410242
20422,4 Series,2017,18200,Automatic,52530,Petrol,48.7,2.0,bmw,29787404
24687,Q5,2023,53000,Automatic,3000,Diesel,42.8,2.0,audi,29754169
18150,Passat,2019,20357,Semi-Auto,25050,Petrol,41.5,1.5,vw,29893999
24066,Polo,2021,19500,Manual,10347,Petrol,54.3,1.0,vw,29843706
4964,5 Series,2011,8995,Automatic,93000,Diesel,54.3,2.0,bmw,29964500
6469,1 Series,2020,20900,Manual,58850,Diesel,54.3,1.5,bmw,29991674
17223,5 Series,2018,23190,Semi-Auto,70562,Diesel,55.4,2.0,bmw,29880980


In [20]:
data["brand"].value_counts()

bmw     9023
audi    8469
vw      8083
Name: brand, dtype: int64

In [21]:
data["fuelType"].value_counts()

Diesel      12544
Petrol      11627
Hybrid        781
Electric      623
Name: fuelType, dtype: int64

In [22]:
data["transmission"].value_counts()

Automatic    10351
Manual       10337
Semi-Auto     4887
Name: transmission, dtype: int64